In [ ]:
import time
from pinecone import Pinecone, ServerlessSpec
import matplotlib.pyplot as plt
import numpy as np
import json
from open_clip import tokenizer, model

In [4]:
pc = Pinecone(
    api_key="pcsk_4AKL3D_8e2Zut6CfAGo6oJ3ZvZydtRd4xHaVvzRYcqHRV7fePJ3AzdXfrXK7iS7L62hQHg"
)

In [ ]:
# pc.delete_index('my-index')

In [5]:
if 'my-index' not in pc.list_indexes().names():
    pc.create_index(
        name='my-index', 
        dimension=512, 
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )
     
    while not pc.describe_index('my-index').index.status['ready']:
        time.sleep(1)
    
    print("Pinecone Index provisioned")
else:
    print("Pinecone Index Already Provisioned")

index = pc.Index("my-index")

Pinecone Index Already Provisioned


In [6]:
import open_clip
import torch
from PIL import Image

model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='openai')
model.eval()

d:\CBC_data_science\Macro_factors\Chatbot\chat_venv\Lib\site-packages\open_clip\factory.py:388: UserWarning: These pretrained weights were trained with QuickGELU activation but the model config does not have that enabled. Consider using a model config with a "-quickgelu" suffix or enable with a flag.
  warnings.warn(


CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-11): 12 x ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((768,), eps=1e-05, elementwise_affine

In [7]:
def image_to_embedding(image_path):
    image = Image.open(image_path).convert("RGB")
    image_input = preprocess(image).unsqueeze(0)

    with torch.no_grad():
        embedding = model.encode_image(image_input).cpu().numpy().flatten()

    return embedding

In [8]:
query_embedding = image_to_embedding("images/plot2.png")  # This should return a vector of size 512

# Ensure the query embedding has the correct length
print(len(query_embedding))  # This should print 512

results = index.query(
    vector=query_embedding.tolist(),
    top_k=3,
    include_values=True
)

print(results)

512
{'matches': [], 'namespace': '', 'usage': {'read_units': 1}}
